In [1]:
import multiprocessing, torch, re
import os.path as osp
import datasets

from transformers import EncoderDecoderModel, BertTokenizerFast,\
    Seq2SeqTrainer, Seq2SeqTrainingArguments, BitsAndBytesConfig, AutoModel, AutoTokenizer, T5ForConditionalGeneration, StoppingCriteria, StoppingCriteriaList
from tokenization_kobert import KoBertTokenizer
from datasets import load_dataset, Dataset
from evaluate import load as load_metric
from tqdm.auto import tqdm
from time import strftime, time, localtime
from os import listdir
from functools import partial
from typing import List
from zipfile import ZipFile

from utils import rouge, load_dataset

torch.cuda.set_device("cuda:0")
print(torch.__version__)

1.12.1+cu113


In [2]:
# model_path = "skt/kogpt2-base-v2"
# tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path, 
#                                                     bos_token='</s>', eos_token='</s>', unk_token='<unk>',
#                                                     pad_token='<pad>', mask_token='<mask>', max_length=512)
# model = GPT2LMHeadModel.from_pretrained(model_path)
# model = BertForMaskedLM.from_pretrained("skt/kobert-base-v1")
# model = AutoModel.from_pretrained('monologg/distilkobert')
# model = BertForMaskedLM.from_pretrained("monologg/kobert-lm")
# tokenizer = KoBertTokenizer.from_pretrained('monologg/distilkobert')
quant8_cfg = BitsAndBytesConfig(
    load_in_8bit=True,
)
quant4_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True
)

# path = "output-12-16-11-03/checkpoint-16000"
path = "output-12-16-01-58/checkpoint-18000"

# model = AutoModel.from_pretrained(path)
model = T5ForConditionalGeneration.from_pretrained(path)
# model_8 = EncoderDecoderModel.from_pretrained("output/checkpoint-72000", quantization_config=quant8_cfg)
# model_4 = EncoderDecoderModel.from_pretrained("output/checkpoint-72000", quantization_config=quant4_cfg)
# tokenizer = BertTokenizerFast.from_pretrained(path)
tokenizer = AutoTokenizer.from_pretrained(path)
# model

In [3]:
# mask_idxs = encoded['input_ids'] == tokenizer.mask_token_id
# print(mask_idxs)
# print(out.logits.shape)
# masks = out.logits[mask_idxs].argmax(dim=-1)
# print(masks)
# print(tokenizer.decode(masks))
# predicted_token = out.logits[0,-1].argmax(-1)
# tokenizer.decode(predicted_token)

In [4]:
# dataset = load_dataset("HuggingFaceH4/ultrachat_200k")
# dataset = load_dataset("wikipedia", "20220301.en")
# dataset = dataset.remove_columns([col for col in dataset.column_names if col != 'text'])
# dataset = load_dataset("bookcorpus", split="train", streaming=True).with_format('torch')
# dataset = load_dataset("bookcorpus", split="train")
# dataset

In [5]:
# def batch_iterator(batch_size=10000):
#     for i in tqdm(range(0, len(dataset['train']), batch_size)):
#         yield dataset['train'][i:i+batch_size]['text']
# if [_dir for _dir in listdir() if "tokenizer" in _dir] != []:
#     latest_tokenizer_path = sorted([_dir for _dir in listdir() if "tokenizer" in _dir])[-1]
#     tokenizer = AutoTokenizer.from_pretrained(latest_tokenizer_path)
# else:
#     # tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2").train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=32_000)
#     tokenizer = AutoTokenizer.from_pretrained("skplanet/dialog-koelectra-small-discriminator").train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=32_000)
#     tokenizer.save_pretrained(get_time_dir())

# tokenizer.pad_token = '<pad>'
# tokenizer.eos_token = '</s>'
# tokenizer.bos_token = '</s>'
# tokenizer.mask_token = '<mask>'
# tokenizer.model_max_length = 32

In [6]:
class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = []):
      StoppingCriteria.__init__(self), 

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, stops = []):
      self.stops = stops
      for i in range(len(stops)):
        self.stops = self.stops[i]

In [7]:
print(f"The max length for the tokenizer is: {tokenizer.model_max_length}")

The max length for the tokenizer is: 1000000000000000019884624838656


In [8]:
def get_time_dir(): return f"{strftime('%m-%d-%H-%M', localtime(time()))}"

In [9]:
from utils import load_iter_train_test
# dataset = load_dataset(tokenizer=tokenizer)
trainset, testset = load_iter_train_test(tokenizer=tokenizer)

In [10]:
# one = dataset['train'][0]
# one

In [11]:
# model.config.decoder_start_token_id = tokenizer.bos_token_id
# model.config.eos_token_id = tokenizer.eos_token_id
# model.config.pad_token_id = tokenizer.pad_token_id

model.config.max_new_tokens = 32
# model.config.early_stopping = True
# model.config.no_repeat_ngram_size = 1
# model.config.length_penalty = 2.
# model.config.repetition_penalty = 3.
# model.config.num_beams = 10
# model.config.vocab_size = model.config.encoder.vocab_size


In [12]:
model.config.max_length

20

In [13]:
tokenizer.eos_token_id, tokenizer.bos_token_id

(32000, 32000)

In [14]:
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops = [tokenizer.eos_token_id, tokenizer.bos_token_id])])

In [15]:
sent = "summarize: "+"진짜 타이트하게 살고 딨"

In [16]:
encoded = tokenizer(sent, return_tensors="pt")

In [17]:
encoded

{'input_ids': tensor([[8449,  164,  156,  156,  144,  161,  152,  169,  148,    2, 5189, 5312,
         5708, 5356, 6657, 8317,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [18]:
out = model.generate(input_ids=encoded['input_ids'], attention_mask=encoded['attention_mask'], stopping_criteria=stopping_criteria)[0]
out = tokenizer.decode(out)

/home1/sizz1997/miniconda3/envs/torch/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [19]:
out = out[:out.find('</s>')]

In [20]:
out

'<unk> 딨 고생을'